<a href="https://colab.research.google.com/github/Yahyawi99/Yahyawi99/blob/main/PFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install unsloth
!pip install transformers datasets accelerate peft bitsandbytes


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.1 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch

# Use a 4-bit quantized LLaMA 2 7B
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Detected that PyTorch and torchvision were compiled with different CUDA major versions. PyTorch has CUDA Version=11.8 and torchvision has CUDA Version=12.4. Please reinstall the torchvision that matches your PyTorch install.

In [32]:
# Clean old versions first
!pip uninstall -y torch torchvision torchaudio
!pip install --upgrade pip

# Install Unsloth and required packages
!pip install unsloth
!pip install transformers datasets accelerate peft bitsandbytes



Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: torchaudio 2.6.0+cu118
Uninstalling torchaudio-2.6.0+cu118:
  Successfully uninstalled torchaudio-2.6.0+cu118
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl (766.7 MB)
Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl (7.2 MB)


In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Llama 3 is up to 8k
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Llama-3 70b also works (just change the model name)
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:


model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [20]:
from datasets import Dataset

# Hardcoded dataset: just input/output pairs
schema = """# Table Schemas and Relationships

-- employees
employee_id INT PRIMARY KEY,
first_name VARCHAR(20),
last_name VARCHAR(25),
email VARCHAR(25),
phone_number VARCHAR(20),
hire_date DATE,
job_id VARCHAR(10),
salary DECIMAL(8,2),
commission_pct DECIMAL(2,2),
manager_id INT,
department_id INT

-- departments
department_id INT PRIMARY KEY,
department_name VARCHAR(30),
manager_id INT,
location_id INT

-- jobs
job_id VARCHAR(10) PRIMARY KEY,
job_title VARCHAR(35),
min_salary DECIMAL(8,2),
max_salary DECIMAL(8,2)

-- locations
location_id INT PRIMARY KEY,
street_address VARCHAR(40),
postal_code VARCHAR(12),
city VARCHAR(30),
state_province VARCHAR(25),
country_id CHAR(2)

-- countries
country_id CHAR(2) PRIMARY KEY,
country_name VARCHAR(40),
region_id INT

-- regions
region_id INT PRIMARY KEY,
region_name VARCHAR(25)

-- job_grades
grade_level CHAR(1) PRIMARY KEY,
lowest_sal DECIMAL(8,2),
highest_sal DECIMAL(8,2)

# Relationships:
employees.department_id → departments.department_id
employees.manager_id → employees.employee_id
employees.job_id → jobs.job_id
departments.manager_id → employees.employee_id
departments.location_id → locations.location_id
locations.country_id → countries.country_id
countries.region_id → regions.region_id
employees.salary → within job_grades.lowest_sal–highest_sal (logical)
"""


data = [
  {
    "input": schema,
    "instruction": "Write a query to get first name and last name of employees working in the Sales department.",
    "output": "SELECT e.first_name, e.last_name FROM employees e INNER JOIN departments d ON e.department_id = d.department_id WHERE d.department_name = 'Sales';"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, and department name of all employees with or without a department.",
    "output": "SELECT e.first_name, e.last_name, d.department_name FROM employees e LEFT JOIN departments d ON e.department_id = d.department_id;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, department name, and city of employees working in the department and located in a city.",
    "output": "SELECT e.first_name, e.last_name, d.department_name, l.city FROM employees e INNER JOIN departments d ON e.department_id = d.department_id INNER JOIN locations l ON d.location_id = l.location_id;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name and last name of employees working in the Marketing department, including those without a department.",
    "output": "SELECT e.first_name, e.last_name FROM employees e RIGHT JOIN departments d ON e.department_id = d.department_id WHERE d.department_name = 'Marketing';"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, and department name for all employees, including those without a department.",
    "output": "SELECT e.first_name, e.last_name, d.department_name FROM employees e FULL OUTER JOIN departments d ON e.department_id = d.department_id;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, and job title of employees who are managers.",
    "output": "SELECT e.first_name, e.last_name, j.job_title FROM employees e LEFT JOIN jobs j ON e.job_id = j.job_id WHERE j.job_title LIKE '%Manager%';"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, department name, and salary of employees who have a salary greater than 50000.",
    "output": "SELECT e.first_name, e.last_name, d.department_name, e.salary FROM employees e INNER JOIN departments d ON e.department_id = d.department_id WHERE e.salary > 50000;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name and last name of employees who do not belong to any department.",
    "output": "SELECT e.first_name, e.last_name FROM employees e LEFT JOIN departments d ON e.department_id = d.department_id WHERE d.department_name IS NULL;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, and department name of employees ordered by their last name.",
    "output": "SELECT e.first_name, e.last_name, d.department_name FROM employees e LEFT JOIN departments d ON e.department_id = d.department_id ORDER BY e.last_name;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, and job title of employees who are Software Engineers.",
    "output": "SELECT e.first_name, e.last_name, j.job_title FROM employees e INNER JOIN jobs j ON e.job_id = j.job_id WHERE j.job_title = 'Software Engineer';"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, salary, and department name of employees with a salary between 40000 and 60000.",
    "output": "SELECT e.first_name, e.last_name, e.salary, d.department_name FROM employees e LEFT JOIN departments d ON e.department_id = d.department_id WHERE e.salary BETWEEN 40000 AND 60000;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, and project name of employees working on projects.",
    "output": "SELECT e.first_name, e.last_name, p.project_name FROM employees e LEFT JOIN employee_projects ep ON e.employee_id = ep.employee_id LEFT JOIN projects p ON ep.project_id = p.project_id;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, and store name of employees working at a specific store.",
    "output": "SELECT e.first_name, e.last_name, s.store_name FROM employees e INNER JOIN stores s ON e.store_id = s.store_id WHERE s.store_name = 'New York Store';"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, project name of employees working on projects, including those with no project.",
    "output": "SELECT e.first_name, e.last_name, p.project_name FROM employees e INNER JOIN employee_projects ep ON e.employee_id = ep.employee_id LEFT JOIN projects p ON ep.project_id = p.project_id WHERE p.project_name IS NOT NULL;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, department name, and city of employees working in a department located in Paris.",
    "output": "SELECT e.first_name, e.last_name, d.department_name, l.city FROM employees e INNER JOIN departments d ON e.department_id = d.department_id LEFT JOIN locations l ON d.location_id = l.location_id WHERE l.city = 'Paris';"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, and store name of employees working at a store located in a specific department.",
    "output": "SELECT e.first_name, e.last_name, s.store_name FROM employees e INNER JOIN stores s ON e.store_id = s.store_id LEFT JOIN departments d ON e.department_id = d.department_id WHERE d.department_name = 'Electronics';"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name and last name of employees from the HR department or Finance department.",
    "output": "SELECT e.first_name, e.last_name FROM employees e LEFT JOIN departments d ON e.department_id = d.department_id WHERE d.department_name = 'Finance' OR d.department_name = 'HR';"
  },
  {
    "input": schema,
    "instruction": "Write a query to get the first name, last name, job title, and department name of employees whose job title includes 'Manager' and they work in the 'Sales' department.",
    "output": "SELECT e.first_name, e.last_name, j.job_title, d.department_name FROM employees e INNER JOIN jobs j ON e.job_id = j.job_id INNER JOIN departments d ON e.department_id = d.department_id WHERE j.job_title LIKE '%Manager%' AND d.department_name = 'Sales';"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, department name, and salary of employees, sorted by salary in descending order.",
    "output": "SELECT e.first_name, e.last_name, d.department_name, e.salary FROM employees e LEFT JOIN departments d ON e.department_id = d.department_id ORDER BY e.salary DESC;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get the first name, last name, and department name of employees who are in the department named 'Human Resources' but have no project assigned.",
    "output": "SELECT e.first_name, e.last_name, d.department_name FROM employees e INNER JOIN departments d ON e.department_id = d.department_id LEFT JOIN employee_projects ep ON e.employee_id = ep.employee_id WHERE d.department_name = 'Human Resources' AND ep.project_id IS NULL;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, job title, and department name of employees whose salary is greater than 60000.",
    "output": "SELECT e.first_name, e.last_name, j.job_title, d.department_name FROM employees e INNER JOIN jobs j ON e.job_id = j.job_id INNER JOIN departments d ON e.department_id = d.department_id WHERE e.salary > 60000;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get the project name and number of employees working on each project.",
    "output": "SELECT p.project_name, COUNT(ep.employee_id) AS employee_count FROM projects p LEFT JOIN employee_projects ep ON p.project_id = ep.project_id GROUP BY p.project_name;"
  },
  {
    "input": schema,
    "instruction": "Write a query to get first name, last name, and the department they work in for employees whose first name starts with 'J'.",
    "output": "SELECT e.first_name, e.last_name, d.department_name FROM employees e INNER JOIN departments d ON e.department_id = d.department_id WHERE e.first_name LIKE 'J%';"
  }
]

# Convert to HuggingFace dataset
dataset = Dataset.from_list(data)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # do not forget this part!
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [21]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 4096,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # increase this to make the model learn "better"
        num_train_epochs=4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/23 [00:00<?, ? examples/s]

In [22]:
# We're now kicking off the actual training of our model, which will spit out some statistics showing us how well it learns
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 23 | Num Epochs = 20 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 39,976,960/7,000,000,000 (0.57% trained)


Step,Training Loss
1,0.179600
2,0.175100
3,0.164800
4,0.081800
5,0.069800
6,0.064100
7,0.058900
8,0.041300
9,0.043600
10,0.040000


In [26]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "The HR department needs a report that displays the last name, department number, and job id of all employees whose department location id is 1700.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nThe HR department needs a report that displays the last name, department number, and job id of all employees whose department location id is 1700.\n\n### Input:\n\n\n### Response:\nSELECT e.last_name, d.department_name, j.job_title FROM employees e INNER JOIN departments d ON e.department_id = d.department_id INNER JOIN jobs j ON e.job_id = j.job_id WHERE d.location_id = 1700;</s>']